In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../code/')

In [3]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

import utils
from data_loading import ImageSequence

In [4]:
tf.get_logger().setLevel('INFO')

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [6]:
from keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy
from keras.optimizers import Adam
from keras import callbacks
from sklearn.utils.class_weight import compute_class_weight

In [12]:
dataset_path = '/data/datasets/imagenet2012/'

- https://www.pyimagesearch.com/2020/10/19/adversarial-images-and-attacks-with-keras-and-tensorflow/
- http://image-net.org/challenges/LSVRC/2014/browse-synsets

In [6]:
resnet = tf.keras.applications.ResNet50(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)


In [8]:
# change last layer to new config  
#
# last_layer_config = resnet.layers[-1].get_config()
# last_layer_config['units'] = n_classes
# last_layer_config['name'] = 'predictions1001'
# new_last_layer = keras.layers.Dense(**last_layer_config)#(model.layers[-2].output)
#
# old_resnet = keras.Model(resnet.input, resnet.layers[-2].output)
# new_model = keras.Model(old_resnet.input, new_last_layer(old_resnet.output))

### Brad model

In [9]:
tr_brad_gen = ImageSequence(dataset_path, 
                            80, 
                            target_split='train', 
                            square_ims='center_pad', 
                            only_brad=True, 
                            augment=True)
val_brad_gen = ImageSequence(dataset_path, 
                             80, 
                             target_split='val',
                             square_ims='center_pad', 
                             only_brad=True, 
                             augment=False)

  1%|          | 12/1000 [00:00<00:08, 119.29it/s]

generating splits
only brad!


  1%|          | 12/1000 [00:00<00:08, 114.33it/s]

total 1 ims: 716
generating splits
only brad!


100%|██████████| 1000/1000 [00:08<00:00, 111.18it/s]

total 1 ims: 716


In [11]:
class_w = compute_class_weight('balanced', [0, 1], np.squeeze([x[0] for x in tr_brad_gen.x_tr]))
class_w = {k:v for k, v in zip([0,1], class_w)}
class_w

val_class_w = compute_class_weight('balanced', [0, 1], np.squeeze([x[0] for x in tr_brad_gen.x_val]))
val_class_w = {k:v for k, v in zip([0,1], val_class_w)}
class_w, val_class_w

/home/ivan/coding_projects/ThisIsNotBradPitt/env/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass classes=[0, 1], y=[0 0 0 ... 1 1 1] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


({0: 0.5895, 1: 3.293296089385475},
 {0: 0.5238333333333334, 1: 10.98951048951049})

In [12]:
class_w = {0: 0.8, 1: 2.1}

In [18]:
last_layer_config = resnet.layers[-1].get_config()
last_layer_config['units'] = 1
last_layer_config['name'] = 'predictions_brad'
last_layer_config['activation'] = 'sigmoid'
new_last_layer = keras.layers.Dense(**last_layer_config)#(model.layers[-2].output)

old_resnet = keras.Model(resnet.input, resnet.layers[-2].output)
for l in old_resnet.layers:
    l.trainable = False
brad_model = keras.Model(old_resnet.input, new_last_layer(old_resnet.output))

In [19]:
opti = Adam(lr=0.001)
loss = BinaryCrossentropy()
brad_model.compile(optimizer=opti, loss=loss)

for l in brad_model.layers[:-1]:
    l.trainable = False

In [20]:
es_cb = callbacks.EarlyStopping(
    monitor='val_loss',
    verbose=True,
    patience=4,
    min_delta=0.001,
    restore_best_weights=True)

lr_cb = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    verbose=True,
    patience=3,
    mi_delta=0.001)

In [21]:
his = brad_model.fit(
    tr_brad_gen,
    epochs=20,
    verbose=1,
    validation_data=val_brad_gen,
    workers=6,
    class_weight=class_w,
    use_multiprocessing=True,
    callbacks = [es_cb, lr_cb])

Epoch 1/20
45/45 [==============================] - 33s 611ms/step - loss: 0.4002 - val_loss: 0.0756
Epoch 2/20
45/45 [==============================] - 29s 599ms/step - loss: 0.0922 - val_loss: 0.0515
Epoch 3/20
45/45 [==============================] - 29s 600ms/step - loss: 0.0781 - val_loss: 0.0524
Epoch 4/20
45/45 [==============================] - 29s 598ms/step - loss: 0.0685 - val_loss: 0.0388
Epoch 5/20
45/45 [==============================] - 29s 603ms/step - loss: 0.0616 - val_loss: 0.0432
Epoch 6/20
45/45 [==============================] - 29s 603ms/step - loss: 0.0468 - val_loss: 0.0313
Epoch 7/20
45/45 [==============================] - 29s 603ms/step - loss: 0.0469 - val_loss: 0.0321
Epoch 8/20
45/45 [==============================] - 29s 591ms/step - loss: 0.0438 - val_loss: 0.0276
Epoch 9/20
45/45 [==============================] - 29s 600ms/step - loss: 0.0374 - val_loss: 0.0249
Epoch 10/20
44/45 [============================>.] - ETA: 0s - loss: 0.0395WARNING:tensorfl

45/45 [==============================] - 29s 585ms/step - loss: 0.0396 - val_loss: 0.0385
Epoch 11/20
45/45 [==============================] - 29s 591ms/step - loss: 0.0418 - val_loss: 0.0300
Epoch 12/20
45/45 [==============================] - 29s 600ms/step - loss: 0.0334 - val_loss: 0.0250

Epoch 00012: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 13/20
45/45 [==============================] - 29s 599ms/step - loss: 0.0352 - val_loss: 0.0261
Restoring model weights from the end of the best epoch.
Epoch 00013: early stopping


Process Keras_worker_ForkPoolWorker-492:
Process Keras_worker_ForkPoolWorker-491:
Process Keras_worker_ForkPoolWorker-488:
Process Keras_worker_ForkPoolWorker-490:
Process Keras_worker_ForkPoolWorker-487:
Process Keras_worker_ForkPoolWorker-489:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ivan/coding_projects/ThisIsNotBradPitt/env/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/ivan/coding_projects/ThisIsNotBradPitt/env/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()


In [22]:
his.history

{'loss': [0.2474277913570404,
  0.09167914092540741,
  0.07671944051980972,
  0.06384716182947159,
  0.05832662060856819,
  0.053900983184576035,
  0.053326740860939026,
  0.046484146267175674,
  0.04038766771554947,
  0.040673937648534775,
  0.03910934552550316,
  0.03501582890748978,
  0.03444567695260048],
 'val_loss': [0.07560590654611588,
  0.05153569206595421,
  0.05235994607210159,
  0.03879997879266739,
  0.04321853816509247,
  0.03127732872962952,
  0.03207918629050255,
  0.027589766308665276,
  0.024884840473532677,
  0.03850669041275978,
  0.029998797923326492,
  0.024955200031399727,
  0.02614044025540352],
 'lr': [0.001,
  0.001,
  0.001,
  0.001,
  0.001,
  0.001,
  0.001,
  0.001,
  0.001,
  0.001,
  0.001,
  0.001,
  0.000100000005]}

In [23]:
pred_test = []
for x in tqdm(val_brad_gen.x_test):
    s = tr_brad_gen.load_im_and_proc(x[1])
    pred_test.append(brad_model.predict(s))
preds = (np.squeeze(pred_test) > 0.5).astype('float32')
y_true = np.squeeze([x[0] for x in val_brad_gen.x_test])    

100%|██████████| 2036/2036 [02:19<00:00, 14.64it/s]


In [24]:
# brad_model.save('./brad_model_with_weights.hdf5')

In [25]:
from sklearn.metrics import classification_report
print(classification_report(y_true, preds))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      2000
           1       0.72      1.00      0.84        36

    accuracy                           0.99      2036
   macro avg       0.86      1.00      0.92      2036
weighted avg       1.00      0.99      0.99      2036



##### use trained weights for last layer in new model

In [26]:
train_generator = ImageSequence(dataset_path, 20, target_split='train', square_ims='center_pad')
n_classes = train_generator.n_classes()

# change last layer to new config  
last_layer_config = resnet.layers[-1].get_config()
last_layer_config['units'] = n_classes
last_layer_config['name'] = 'predictions1001'
new_last_layer = keras.layers.Dense(**last_layer_config)#(model.layers[-2].output)

old_resnet = keras.Model(resnet.input, resnet.layers[-2].output)
for l in old_resnet.layers:
    l.trainable = False
new_model = keras.Model(old_resnet.input, new_last_layer(old_resnet.output))

  1%|          | 11/1001 [00:00<00:09, 105.49it/s]

generating splits


100%|██████████| 1001/1001 [00:10<00:00, 97.74it/s] 


In [27]:
l = resnet.layers[-1]
lw, lb = l.get_weights()
print('resnet w, b')
print(lw.shape, lb.shape)

brad_l = brad_model.layers[-1]
blw, blb = brad_l.get_weights()
print('brad w, b')
print(blw.shape, blb.shape)

new_w = np.concatenate([lw, blw], axis=1)
new_b = np.concatenate([lb, blb])
print('new w, b')
print(new_w.shape, new_b.shape)
assert new_w.shape[1] == n_classes
assert new_b.shape[0] == n_classes

new_model.layers[-1].set_weights([new_w, new_b])
setted_w = new_model.layers[-1].get_weights()
assert np.all(setted_w[0] == new_w)
assert np.all(setted_w[1] == new_b)
print('weights and biases: OK!')

resnet w, b
(2048, 1000) (1000,)
brad w, b
(2048, 1) (1,)
new w, b
(2048, 1001) (1001,)
weights and biases: OK!


In [28]:
new_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
____________________________________________________________________________________________

## test here metrics of new model on test set, I wonder if they are similar to 



In [29]:
from tensorflow.keras.applications.resnet50 import decode_predictions

In [91]:
s = train_generator.get_random_sample('test', cat=1000)
proc_s = train_generator.load_im_and_proc(s[1])
s

[1000, '/data/datasets/imagenet2012//brad/998.jpeg']

In [92]:
pp = new_model.predict(proc_s)
np.argmax(pp[0]), np.max(pp[0]), pp[0][1000]

(916, 0.1837024, 0.0032308116)

In [93]:
pp_rn = resnet.predict(proc_s)
np.argmax(pp_rn[0]), np.max(pp_rn[0]), train_generator.get_id_for_class(np.argmax(pp_rn[0]))

(916, 0.18429784, 'n06359193')

In [94]:
preds_codes = decode_predictions(pp_rn, top=3)
preds_codes

[[('n06359193', 'web_site', 0.18429784),
  ('n03291819', 'envelope', 0.07401998),
  ('n06596364', 'comic_book', 0.03699455)]]

## Retrain

In [96]:
opti = Adam(lr=0.001, clipnorm=1)
loss = SparseCategoricalCrossentropy()
new_model.compile(optimizer=opti, loss=loss)

In [7]:
train_generator = ImageSequence(dataset_path, 80, target_split='train', square_ims='center_pad', augment=True)
val_generator = ImageSequence(dataset_path, 80, target_split='val', square_ims='center_pad', augment=False)

  1%|          | 9/1001 [00:00<00:11, 89.04it/s]

generating splits


  1%|          | 12/1001 [00:00<00:08, 118.43it/s]

generating splits


100%|██████████| 1001/1001 [00:09<00:00, 108.23it/s]


##### First we are going to retrain only last layers

In [98]:
for l in new_model.layers[:-1]:
    l.trainable = False

In [99]:
his_full = new_model.fit(
    train_generator,
    epochs=10,
    verbose=1,
    validation_data=val_generator,
    workers=6,
    use_multiprocessing=True,
    callbacks = [es_cb, lr_cb])

Epoch 1/10
4504/7089 [==================>...........] - ETA: 30:28 - loss: 1.5956

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



5257/7089 [=====================>........] - ETA: 20:08 - loss: 1.4651

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



6555/7089 [==========================>...] - ETA: 6:13 - loss: 1.4038

...

7089/7089 [==============================] - 6456s 910ms/step - loss: 0.6951 - val_loss: 1.2940
Epoch 9/10
 293/7089 [>.............................] - ETA: 1:09:32 - loss: 0.6355

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



7089/7089 [==============================] - 6447s 909ms/step - loss: 0.6582 - val_loss: 1.2809
Epoch 10/10
 417/7089 [>.............................] - ETA: 1:09:29 - loss: 0.6506

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



7089/7089 [==============================] - 6438s 908ms/step - loss: 0.6460 - val_loss: 1.2594


In [103]:
his_full.history

{'loss': [1.641484022140503,
  1.5368335247039795,
  1.4638113975524902,
  1.4175344705581665,
  0.8634382486343384,
  0.7648922801017761,
  0.7228109240531921,
  0.6942424774169922,
  0.6669949293136597,
  0.6522877812385559],
 'val_loss': [1.5938208103179932,
  1.688230276107788,
  1.7797939777374268,
  1.8615033626556396,
  1.4031524658203125,
  1.3496508598327637,
  1.3153696060180664,
  1.293982744216919,
  1.2808637619018555,
  1.259441614151001],
 'lr': [0.001,
  0.001,
  0.001,
  0.001,
  0.000100000005,
  0.000100000005,
  0.000100000005,
  0.000100000005,
  0.000100000005,
  0.000100000005]}

In [ ]:
# new_model.save('./resnet_1001_10_iters.hdf5')

In [14]:
train_generator = ImageSequence(dataset_path, 80, target_split='train', square_ims='center_pad', augment=True)
val_generator = ImageSequence(dataset_path, 80, target_split='val', square_ims='center_pad', augment=False)

  1%|          | 12/1001 [00:00<00:08, 117.78it/s]

generating splits


  1%|          | 12/1001 [00:00<00:08, 112.95it/s]

generating splits


100%|██████████| 1001/1001 [00:09<00:00, 104.12it/s]


In [7]:
new_model = keras.models.load_model('./resnet_1001_10_iters.hdf5')

In [8]:
opti = Adam(lr=0.001)
loss = SparseCategoricalCrossentropy()
new_model.compile(optimizer=opti, loss=loss)

In [9]:
for l in new_model.layers[:-1]:
    l.trainable = False

In [13]:
# his_full = new_model.fit(
#     train_generator,
#     epochs=2,
#     verbose=1,
#     validation_data=val_generator,
#     workers=6,
#     use_multiprocessing=True)

In [15]:
from tensorflow.keras.applications.resnet50 import decode_predictions

In [24]:
s = train_generator.get_random_sample('test', cat=1000)
proc_s = train_generator.load_im_and_proc(s[1])
s

[1000, '/data/datasets/imagenet2012//brad/999.jpeg']

In [25]:
pp = new_model.predict(proc_s)
np.argmax(pp[0]), np.max(pp[0]), pp[0][1000]

(1000, 0.14922349, 0.14922349)

In [93]:
pp_rn = resnet.predict(proc_s)
np.argmax(pp_rn[0]), np.max(pp_rn[0]), train_generator.get_id_for_class(np.argmax(pp_rn[0]))

(916, 0.18429784, 'n06359193')

In [94]:
preds_codes = decode_predictions(pp_rn, top=3)
preds_codes

[[('n06359193', 'web_site', 0.18429784),
  ('n03291819', 'envelope', 0.07401998),
  ('n06596364', 'comic_book', 0.03699455)]]

In [ ]:
for l in new_model.layers[:-1]:
    l.trainable = True

In [ ]:
his2 = new_model.fit(
    train_generator,
    epochs=6,
    verbose=1,
    validation_data=val_generator,
    workers=6,
    use_multiprocessing=True,
    callbacks = [es_cb, lr_cb])

In [20]:
his

NameError: name 'his' is not defined

In [ ]:
break

In [ ]:
from PIL import Image

In [ ]:
Image.open('/data/datasets/imagenet2012//n03933933/383.jpeg')

In [ ]:
np.random.shuffle([1,2,3])

In [ ]:
# im = plt.imread('../brad_samples/2097157039_872012429d.jpg')
# new_im = make_square(im)
# plt.figure()
# plt.imshow(new_im)

In [ ]:
import os

In [ ]:
# imgs_path = '/data/datasets/imagenet2012/n02319095/' #sea_urchin

valid_imgs = [x for x in os.listdir(imgs_path) if x.split('.')[-1] != 'json']

In [ ]:
# imgs_path = '/data/datasets/imagenet2012/n02319095/'
# 
# ims = []
# for i in valid_imgs:
#     i_path = '{}/{}'.format(imgs_path,i)
#     im_ar =  image.img_to_array(image.load_img(i_path, target_size=(224, 224)))
#     ims.append(im_ar)
# # img = 
# # x = image.img_to_array(img)
# # x = np.expand_dims(x, axis=0)
# x = np.array(ims)
# x = preprocess_input(x)

# preds = model.predict(x)
# # decode the results into a list of tuples (class, description, probability)
# # (one such list for each sample in the batch)
# print('Predicted:', decode_predictions(preds, top=10)[0])

In [ ]:
# orig_pred = decode_predictions(preds, top=5)
# for pred in orig_pred:
#     print(pred)

In [ ]:
preds_codes = decode_predictions(np.expand_dims(np.zeros(1000), axis=0), top=1000)[0]

In [ ]:
preds_codes

In [ ]:
from PIL import Image

In [ ]:
# vi = valid_imgs[0]
# i_path = '{}/{}'.format(imgs_path,vi)
# i_path = '/home/ivan/Dropbox/scratch/hog_test.png'
i_path = '/home/ivan/scratch/marmot.jpg'
# i_path = '/home/ivan/scratch/guitarra_electrica.jpg'
# ii = image.load_img(i_path, target_size=(224, 224), )
im_pil_raw_squared = center_crop_square(i_path, desired_size=224)
im_ar =  image.img_to_array(im_pil_raw_squared).copy()
im_ar_prec = preprocess_input(np.expand_dims(im_ar.copy(), axis=0))[0]

In [ ]:
plt.figure()
plt.imshow(im_ar/255)
plt.figure()
plt.imshow(im_ar_prec)
plt.figure()
plt.imshow(depreprocess(im_ar_prec)/255)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy

https://www.tensorflow.org/tutorials/generative/adversarial_fgsm

In [ ]:
EPS = 3 / 255.0
LR = 0.1
# EPS = 0.001
# LR = 0.05
optimizer = Adam(learning_rate=LR)
# lossfn = SparseCategoricalCrossentropy()
lossfn = CategoricalCrossentropy()

def clip_eps(tensor, eps):
    # clip the values of the tensor to a given range and return it
    return tf.clip_by_value(tensor, clip_value_min=-eps,
        clip_value_max=eps)

choices = 3

def generate_adversaries(model, baseImage, delta, classIdx, steps=50, verbose=False):
    ty = np.zeros(1000)
    ty[classIdx] = 1
#    # multi labels
#     aa = np.random.choice(np.arange(1000),choices)
#     for a in aa:
#         ty[a] = 1/choices
    ty = np.expand_dims(ty, axis=0)
    for step in range(0, steps):
        with tf.GradientTape() as tape:
            tape.watch(delta)
            adversary = baseImage + preprocess_input(delta)
            predictions = model(adversary, training=False)
            loss = lossfn(tf.convert_to_tensor(ty), predictions)                        
            if verbose and step % 10 == 0:
                print("step: {}, loss: {}...".format(step,
                    loss.numpy()))
        # calculate the gradients of loss with respect to the perturbation vector
        gradients = tape.gradient(loss, delta)
        # update the weights, clip the perturbation vector, and
        # update its value
        optimizer.apply_gradients([(gradients, delta)])
        delta.assign_add(clip_eps(delta, eps=EPS))
    return delta            

In [ ]:
# ii = image.load_img(i_path, target_size=(224, 224))
# ii = image.load_img(i_path, target_size=(224, 224))
im_ar =  image.img_to_array(im_pil_raw_squared).copy()
im_proc = preprocess_input(np.expand_dims(im_ar.copy(), axis=0))

baseImage = tf.constant(im_proc, dtype=tf.float32)
delta = tf.Variable(tf.zeros_like(baseImage), trainable=True)
deltaUpdated = generate_adversaries(model, baseImage, delta, 245, steps=200, verbose=True)

adverImage = (baseImage + deltaUpdated).numpy().squeeze()
adverImage = np.clip(adverImage, 0, 255).astype("uint8")

In [ ]:
ims_test = [baseImage, deltaUpdated]
for i in range(10):
    ims_test.append(baseImage + i* deltaUpdated)

ims_test = np.concatenate(ims_test,axis=0)
preds_all = model.predict(ims_test)

for ii, p in zip(ims_test, decode_predictions(preds_all, top=3)):
    plt.figure()
    plt.imshow(np.round(depreprocess(ii)).astype('int32'))
    print(p)
    plt.show()
    print('______________')

### Brute force loop

In [ ]:
im_ar =  image.img_to_array(im_pil_raw_squared).copy()
im_proc = preprocess_input(np.expand_dims(im_ar.copy(), axis=0))

noise_strengs = np.arange(10) / 2
# iterate over different noise categories
outputs = []
for noise_cat in tqdm(np.random.choice(np.arange(1000), 4, replace=True)):

    baseImage = tf.constant(im_proc, dtype=tf.float32)
    delta = tf.Variable(tf.zeros_like(baseImage), trainable=True)
#     print("[INFO] generating perturbation...")
    deltaUpdated = generate_adversaries(model, baseImage, delta, noise_cat, steps=300, verbose=False)

    adverImage = (baseImage + deltaUpdated).numpy().squeeze()
    adverImage = np.clip(adverImage, 0, 255).astype("uint8")

#     ims_test = [baseImage, deltaUpdated]
#     for ns in noise_strengs:
#         ims_test.append(baseImage + ns * deltaUpdated)
    ims_test = np.concatenate([baseImage + ns * deltaUpdated for ns in noise_strengs], axis=0)
    preds_all = model.predict(ims_test)
    preds_decoded = decode_predictions(preds_all, top=3)
    for ii, p in zip(ims_test, preds_decoded):
        plt.figure()
        plt.imshow(np.round(depreprocess(ii)).astype('int32'))
        print(p)
        plt.show()
        print('______________')
    
    for ns, pred in zip(noise_strengs, preds_decoded):
        sample = [i_path, noise_cat, ns, pred]
        outputs.append(sample)
        
            

In [ ]:
outputs